In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.linear_model import LinearRegression

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리

In [ ]:
# 데이테 불러오기
noplan = pd.read_csv('/content/drive/MyDrive/2023_동계방학_빅데이터 활용 미래사회 문제 예측/활용 데이터/자녀 계획  없는 이유/_자녀_계획_없음_응답자_자녀_계획_없는_이유_격년__20240112003531.csv', encoding='cp949')
noplan.head(3)

,구분별(1),구분별(2),2022,2022.1,2022.2,2022.3,2022.4,2022.5,2022.6,2022.7,2022.8,2022.9,2022.10,2022.11
0,구분별(1),구분별(2),여성,여성,여성,여성,여성,여성,남성,남성,남성,남성,남성,남성
1,구분별(1),구분별(2),연령 건강상태 등으로 인해,이미 가족계획이 완료되어서,경제적인 이유로,자녀를 돌볼 사람이 없어서,직장 일이 바빠서,기타,연령 건강상태 등으로 인해,이미 가족계획이 완료되어서,경제적인 이유로,자녀를 돌볼 사람이 없어서,직장 일이 바빠서,기타
2,전체,소계,27.3,44.9,6.6,8.1,3.1,9.9,18.4,56.4,13.5,6.3,1.3,4.2


In [ ]:
# 열 이름 변경
noplan.columns = ['구분별(1)', '구분별(2)', '연령 및 건강상태_여성', '가족계획 완료_여성', '경제적_여성', '자녀돌봄 어려움_여성', '직장 일_여성', '기타_여성',
                                            '연령 및 건강상태_남성', '가족계획 완료_남성', '경제적_남성', '자녀돌봄 어려움_남성', '직장 일_남성', '기타_남성']

re_noplan = noplan.drop([0], axis = 0)
re_noplan.head(3)

,구분별(1),구분별(2),연령 및 건강상태_여성,가족계획 완료_여성,경제적_여성,자녀돌봄 어려움_여성,직장 일_여성,기타_여성,연령 및 건강상태_남성,가족계획 완료_남성,경제적_남성,자녀돌봄 어려움_남성,직장 일_남성,기타_남성
1,구분별(1),구분별(2),연령 건강상태 등으로 인해,이미 가족계획이 완료되어서,경제적인 이유로,자녀를 돌볼 사람이 없어서,직장 일이 바빠서,기타,연령 건강상태 등으로 인해,이미 가족계획이 완료되어서,경제적인 이유로,자녀를 돌볼 사람이 없어서,직장 일이 바빠서,기타
2,전체,소계,27.3,44.9,6.6,8.1,3.1,9.9,18.4,56.4,13.5,6.3,1.3,4.2
3,규모,100~199인,34.6,45.2,6.0,5.0,1.4,7.8,19.3,50.0,17.5,7.4,1.3,4.5


In [ ]:
# 여성 데이터만 추출
noplan_woman = re_noplan[['구분별(1)', '구분별(2)'] + [col for col in re_noplan.columns if '여성' in col]]
noplan_woman['성별'] = '여성'
new_columns = {'구분별(1)' : '구분별(1)', '구분별(2)':'구분별(2)', '연령 및 건강상태_여성' : '연령 및 건강상태', '가족계획 완료_여성':'가족계획 구성 완료', '경제적_여성': '경제적', '자녀돌봄 어려움_여성':'자녀돌봄 어려움', '직장 일_여성': '직장 일', '기타_여성':'기타'}

noplan_woman = noplan_woman.rename(columns=new_columns)
noplan_woman_drop = noplan_woman.dropna(axis=0)
noplan_woman_drop.head(3)

<ipython-input-43-6988089b7c2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noplan_woman['성별'] = '여성'


,구분별(1),구분별(2),연령 및 건강상태,가족계획 구성 완료,경제적,자녀돌봄 어려움,직장 일,기타,성별
1,구분별(1),구분별(2),연령 건강상태 등으로 인해,이미 가족계획이 완료되어서,경제적인 이유로,자녀를 돌볼 사람이 없어서,직장 일이 바빠서,기타,여성
2,전체,소계,27.3,44.9,6.6,8.1,3.1,9.9,여성
3,규모,100~199인,34.6,45.2,6.0,5.0,1.4,7.8,여성


In [ ]:
# 남성 데이터만 추출
noplan_man = re_noplan[['구분별(1)', '구분별(2)'] + [col for col in re_noplan.columns if '남성' in col]]
noplan_man['성별'] = '남성'
new_columns = {'구분별(1)' : '구분별(1)', '구분별(2)':'구분별(2)', '연령 및 건강상태_남성' : '연령 및 건강상태', '가족계획 완료_남성':'가족계획 구성 완료', '경제적_남성': '경제적', '자녀돌봄 어려움_남성':'자녀돌봄 어려움', '직장 일_남성': '직장 일', '기타_남성':'기타'}

noplan_man = noplan_man.rename(columns=new_columns)
noplan_man_drop = noplan_man.dropna(axis=0)
noplan_man_drop.head(3)

<ipython-input-44-6c7d0dd86225>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noplan_man['성별'] = '남성'


,구분별(1),구분별(2),연령 및 건강상태,가족계획 구성 완료,경제적,자녀돌봄 어려움,직장 일,기타,성별
1,구분별(1),구분별(2),연령 건강상태 등으로 인해,이미 가족계획이 완료되어서,경제적인 이유로,자녀를 돌볼 사람이 없어서,직장 일이 바빠서,기타,남성
2,전체,소계,18.4,56.4,13.5,6.3,1.3,4.2,남성
3,규모,100~199인,19.3,50.0,17.5,7.4,1.3,4.5,남성


In [ ]:
# 데이터 통합
new_noplan = pd.concat([noplan_woman_drop, noplan_man_drop])
new_noplan.head(3)

,구분별(1),구분별(2),연령 및 건강상태,가족계획 구성 완료,경제적,자녀돌봄 어려움,직장 일,기타,성별
1,구분별(1),구분별(2),연령 건강상태 등으로 인해,이미 가족계획이 완료되어서,경제적인 이유로,자녀를 돌볼 사람이 없어서,직장 일이 바빠서,기타,여성
2,전체,소계,27.3,44.9,6.6,8.1,3.1,9.9,여성
3,규모,100~199인,34.6,45.2,6.0,5.0,1.4,7.8,여성


In [ ]:
# 필요없는 행 삭제
last_noplan = new_noplan.drop([1], axis = 0)
last_noplan.head(100)

,구분별(1),구분별(2),연령 및 건강상태,가족계획 구성 완료,경제적,자녀돌봄 어려움,직장 일,기타,성별
2,전체,소계,27.3,44.9,6.6,8.1,3.1,9.9,여성
3,규모,100~199인,34.6,45.2,6.0,5.0,1.4,7.8,여성
4,규모,200~299인,27.1,37.2,9.9,9.0,4.3,12.4,여성
5,규모,300~499인,25.6,43.5,4.5,11.9,6.8,7.7,여성
6,규모,500인 이상,23.8,47.0,6.5,8.7,2.8,11.1,여성
7,직급,과장급(이하),24.4,42.2,8.2,8.7,3.6,13.0,여성
8,직급,차장급,27.4,49.9,5.9,7.5,3.2,6.2,여성
9,직급,부장급,39.4,43.6,2.0,7.3,1.0,6.7,여성
10,직급,임원급,40.1,33.4,2.4,14.5,-,9.6,여성
11,업종,농업 광업 제조업,23.1,46.8,4.8,12.5,4.0,8.8,여성
